In [ ]:

# 1. IMPORT LIBRARIES

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries imported successfully!")


# 2. LOAD THE DATASET
df = pd.read_csv('books.csv', on_bad_lines='skip')
print("Dataset loaded successfully.")


# 3. CLEAN THE DATA (THE FINAL FIX)


prices_no_symbol = df['price'].str.replace('Â£', '')



df['price'] = pd.to_numeric(prices_no_symbol, errors='coerce')
print("'price' column has been successfully cleaned!")


# 4. VERIFY THE FIX

print("\n--- Data Info After Cleaning ---")

df.info()


missing_prices = df['price'].isnull().sum()
print(f"\nNumber of rows with missing/unconvertible prices: {missing_prices}")


print("\n--- Cleaned Data Head ---")
display(df.head())

In [ ]:

# 5. VISUALIZE THE DATA TO FIND INSIGHTS


print("--- Creating Visualizations ---")


plt.figure(figsize=(18, 8))

# Plot 1: What is the distribution of prices? 
plt.subplot(1, 2, 1) # (1 row, 2 columns, 1st plot)
sns.histplot(df['price'], bins=30, kde=True)
plt.title('Distribution of Book Prices', fontsize=16)
plt.xlabel('Price (£)', fontsize=12)
plt.ylabel('Number of Books', fontsize=12)

#  Plot 2: Is there a relationship between rating and price? 
plt.subplot(1, 2, 2) 
rating_order = ['One', 'Two', 'Three', 'Four', 'Five']
sns.boxplot(data=df, x='rating', y='price', order=rating_order)
plt.title('Price Distribution by Book Rating', fontsize=16)
plt.xlabel('Rating', fontsize=12)
plt.ylabel('Price (£)', fontsize=12)


plt.tight_layout()
plt.show()

#  Finding the Top 10 Most Expensive Books 
print("\n--- Top 10 Most Expensive Books ---")
most_expensive_books = df.sort_values(by='price', ascending=False).head(10)
display(most_expensive_books[['title', 'price', 'rating']])

## Database Integration & SQL Insights

In [ ]:
import pandas as pd
import psycopg2 # Import the raw driver


DB_USER = "kushsoni"
DB_PASS = "whitedevil999" 
DB_HOST = "localhost"
DB_NAME = "bookstore_db"
# CRITICAL: This password has been removed for security before committing.
DB_PASS = "YOUR_PASSWORD_REMOVED"


sql_query = """
SELECT
    rating,
    COUNT(title) AS book_count,
    ROUND(AVG(price)::numeric, 2) AS average_price
FROM
    books_data
GROUP BY
    rating
ORDER BY
    average_price DESC;
"""

try:
    # 1. Establish a raw connection using the psycopg2 driver
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS
    )

    # 2. Pass the raw connection to pandas to execute the query
    df_sql_insights = pd.read_sql(sql_query, con=conn)
    
    # 3. Close the connection
    conn.close()

    print("SQL Insights: Average Price and Count by Rating (from PostgreSQL)")
    print(df_sql_insights.to_markdown(index=False)) 

except Exception as e:
    print(f"ERROR connecting or querying the database: {e}")
    # This check is still useful in case of a table name error
    print("Check: Is the table named 'books_data'?")